# Data Preprocessing and Cleaning

Data cleaning is an essential step in the process of preparing data for further processing and serving into a large language model. Without data cleaning, the language model would be limited in its ability to interpret and process the data accurately. Data cleaning helps to ensure that the data is in a consistent format, free of any inconsistencies or errors that could lead to incorrect results. It also helps to reduce the amount of time needed to process the data, as it eliminates the need to manually check for any inconsistencies or errors. Additionally, data cleaning can help to improve the accuracy of the language model by removing any irrelevant or incorrect data. By taking the time to properly clean the data, the language model can more accurately interpret and process the data, leading to better results.

> 📍 Fill out the missing pieces in the source source to get everything working (indicated by `#FIXME`).

## Count tokens

> **Important note**
>
> *You do not need to manually tokenize strings before feeding texts into the model. This will be done automatically once you put instructions into the `prompt` parameter. However, you can use the `tiktoken` library to check how a string is tokenized and count the numbers of tokens to calculate the cost of an API call. Learn more [here](https://platform.openai.com/docs/introduction/tokens).*

Tokenizing text strings is an important step in natural language processing (NLP) as it helps models like GPT-3 understand the structure of a text string. Tokenizing breaks a text string into smaller pieces called tokens, which can then be analyzed and used by the model. By understanding the structure of a text string, models can better understand the meaning of the text. Additionally, tokenizing helps to determine the cost of an Azure OpenAI Service API call, as usage is priced by token. Furthermore, different models use different encodings, so it is important to tokenize text strings in the appropriate format.

`tiktoken` supports three encodings used by Azure OpenAI Service models:

| Encoding name | Azure OpenAI Service models |
| ------------- | -------------- |
| gpt2 (or r50k_base) | Most GPT-3 models |
| p50k_base | Code models, text-davinci-002, text-davinci-003 |
| cl100k_base | text-embedding-ada-002 |

Tokens in English typically range from one character to one word (e.g. "t" or "great"), though some languages may have tokens that are shorter or longer than one character or word. Spaces are usually placed at the beginning of words (e.g. " is" instead of "is " or "+"is"). You can use the Tokenizer to quickly check how a string is tokenized.

To show it briefly, we will use `tiktoken` to tokenize a text string and see how the output looks like.

In [1]:
import tiktoken

encoding = tiktoken.get_encoding("p50k_base")
encoding.encode("Hello world, this is fun!")

[15496, 995, 11, 428, 318, 1257, 0]

Write a script that shows the string tokens from an input phrase.

In [7]:
#FIXME
encoding.decode([15496, 995, 11, 428, 318, 1257, 0])

def str_token(token):
    return encoding.decode([token])

[str_token(t) for t in [15496, 995, 11, 428, 318, 1257, 0]]

['Hello', ' world', ',', ' this', ' is', ' fun', '!']

Let's write a function to count the number of tokens in a text string.

In [8]:
def get_num_tokens_from_string(string: str, encoding_name: str='p50k_base') -> int:
    """Returns the number of tokens in a string using a given encoding."""
    tokenizer = tiktoken.get_encoding(encoding_name)
    return len(encoding.encode(string))

get_num_tokens_from_string("Hello World, this is fun!")

7

## Clean data

Next we'll perform some light data cleaning by removing redundant whitespace and cleaning up the punctuation to prepare the data for tokenization. Again, tokenization is not required for the model to work, but it is a good practice to do so to ensure that the data is in a consistent format and that the model is able to process the data correctly. Also, it makes sure that the request is not too long for the model as the maximum number of tokens for davinci is 2048, e.g., equivalent to around 2-3 pages of text.

> **Best Practices**
> - **Replace newlines with a single space**: Unless you're embedding code, we suggest replacing newlines (\n) in your input with a single space, as we have observed inferior results when newlines are present.

In [9]:
import re

def normalize_text(string, sep_token = " \n "):
    """Normalize text by removing unnecessary characters and altering the format of words."""
    # make text lowercase
    string = re.sub(r'\s+',  ' ', string).strip()
    string = re.sub(r". ,","",string)
    # remove all instances of multiple spaces
    string = string.replace("..",".")
    string = string.replace(". .",".")
    string = string.replace("\n", "")
    string = string.strip()
    return string

Generate some data to test the cleaning function.

In [12]:
import openai
import os
from dotenv import load_dotenv

load_dotenv()
openai.api_type = "azure"
openai.api_base = os.environ.get("OPENAI_API_BASE")
openai.api_key = os.environ.get("OPENAI_API_KEY")
openai.api_version = "2022-12-01"

# Define embedding model and encoding
EMBEDDING_MODEL = 'text-embedding-ada-002'
COMPLETION_MODEL = 'text-davinci-003'
encoding = tiktoken.get_encoding('cl100k_base')

In [17]:
response = openai.Completion.create(engine="text-davinci-003",
                                    prompt="Generate 3 sentences about a topic of your choice.",
                                    temperature=0.7,
                                    max_tokens=500,
                                    top_p=1,)

In [26]:
response['choices'][0]['text']

'\n\n1. Gardening is a great way to get outside and enjoy the beauty of nature.\n2. Growing your own fruits and vegetables is an incredibly rewarding experience.\n3. Taking the time to tend to your garden is beneficial for both your physical and mental health.'

In [25]:
normalize_text(response['choices'][0]['text']) #FIXME

'1. Gardening is a great way to get outside and enjoy the beauty of nature. 2. Growing your own fruits and vegetables is an incredibly rewarding experience. 3. Taking the time to tend to your garden is beneficial for both your physical and mental health.'